<a href="https://colab.research.google.com/github/dewiginting123/MachineLearning/blob/main/WEEK%209/Week__9_Chapter_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Concise Implementation of Linear Regression

In [1]:
!pip install -U mxnet-cu101==1.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███████████████████████████████▌| 834.1 MB 1.2 MB/s eta 0:00:11tcmalloc: large alloc 1147494400 bytes == 0x3a4bc000 @  0x7f532c50a615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 846.0 MB 19 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [2]:
!pip install d2l==0.17.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.6 MB/s 


In [3]:
from mxnet import autograd, gluon, np, npx
from d2l import mxnet as d2l

npx.set_np()

true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

Reading Dataset

In [4]:
def load_array(data_arrays, batch_size, is_train=True):  #
    """Construct a Gluon data iterator."""
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [5]:
next(iter(data_iter))

[array([[ 1.4661262 , -2.7579627 ],
        [-0.11668406, -1.9449775 ],
        [-0.4984026 , -0.04614415],
        [-0.46041864,  1.8027116 ],
        [ 1.2390957 ,  2.173063  ],
        [ 0.89611703,  0.27472666],
        [ 0.25685814, -3.1371417 ],
        [ 0.3900274 ,  0.27706102],
        [ 0.71535987,  1.8271433 ],
        [-0.23753072, -0.6853332 ]]), array([[16.505901  ],
        [10.579725  ],
        [ 3.3562608 ],
        [-2.8433263 ],
        [-0.7127881 ],
        [ 5.0518103 ],
        [15.36906   ],
        [ 4.0488515 ],
        [-0.57446426],
        [ 6.056817  ]])]

Defining the Model

In [6]:
# `nn` is an abbreviation for neural networks
from mxnet.gluon import nn

net = nn.Sequential()
net.add(nn.Dense(1))

Initializing Model Parameters

In [7]:
from mxnet import init

net.initialize(init.Normal(sigma=0.01))

Defining the Loss Function

In [8]:
loss = gluon.loss.L2Loss()

Defining the Optimization Algorithm

In [9]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

Training

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l.mean().asnumpy():f}')

epoch 1, loss 0.025212
epoch 2, loss 0.000090
epoch 3, loss 0.000051


In [11]:
w = net[0].weight.data()
print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
b = net[0].bias.data()
print(f'error in estimating b: {true_b - b}')

error in estimating w: [0.00021386 0.00029969]
error in estimating b: [0.00029039]
